In [1]:
import pandas

In [2]:
jobs = pandas.read_csv("US_ExemptionStatus.csv")

In [3]:
jobs.columns

Index(['Management Level', 'Job Family Group', 'Job Family', 'Sales Indicator',
       'Job Category', 'Exempt Status'],
      dtype='object')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Define a dictionary to store label encoders and mappings
label_encoders = {}
label_encoder_mappings = {}

# Encode non-numerical variables and store label encoders and mappings
for column in jobs.columns:
    if jobs[column].dtype == 'object':
        label_encoders[column] = LabelEncoder()
        jobs[column] = label_encoders[column].fit_transform(jobs[column])

        # Storing the mapping between original categories and encoded values
        label_encoder_mappings[column] = dict(zip(label_encoders[column].classes_, label_encoders[column].transform(label_encoders[column].classes_)))
        print(dict(zip(label_encoders[column].classes_, label_encoders[column].transform(label_encoders[column].classes_))))
# Define features (X) and target (y)
X = jobs.drop('Exempt Status', axis=1)
y = jobs['Exempt Status']

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the decision tree model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict Exempt Status on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the decision tree model:", accuracy)

# Reverse the encoding using the label encoders
reverse_label_encoders = {col: {label: encoded for encoded, label in mapping.items()} for col, mapping in label_encoder_mappings.items()}


Accuracy of the decision tree model: 0.9363636363636364


In [12]:
# Predict Exempt Status on the train set to see how much overfitting we have had
y_pred_train = model.predict(X_train)

# Calculate the accuracy of the model
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Accuracy of the decision tree model on the train data:", accuracy_train)

Accuracy of the decision tree model on the train data: 0.9863636363636363


In [19]:
#Now training a logistic regression
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model
logistic_regression_model = LogisticRegression(random_state=42)
logistic_regression_model.fit(X_train, y_train)

# Predict Exempt Status on the test set using logistic regression
y_pred_logistic_regression = logistic_regression_model.predict(X_test)

# Calculate the accuracy of the logistic regression model
accuracy_logistic_regression = accuracy_score(y_test, y_pred_logistic_regression)
print("Accuracy of the logistic regression model:", accuracy_logistic_regression)

Accuracy of the logistic regression model: 0.825


C:\Users\Jackson_Aquino\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
#Now looking for signs of overfitting on the log regression, which I imagine there will not be as much as on the decision tree
y_pred_logModel_train = logistic_regression_model.predict(X_train)

accuracy_logModel_train = accuracy_score(y_train, y_pred_logModel_train)
print("Accuracy of the logistic regression model on the training data:", accuracy_logModel_train)

Accuracy of the logistic regression model on the training data: 0.8670454545454546


In [37]:
#Putting them all together to see how they relate to each other
decisionTreeResult = list(y_pred)
logRegressionResult = list(y_pred_logistic_regression)
TrueY = list(y_test)


result_df = pandas.DataFrame({'Decision Tree Result': decisionTreeResult,
                          'Logistic Regression Result': logRegressionResult,
                          'True Y': TrueY})

In [39]:
result_df.to_csv("test.csv",index=False)